In [1]:
include("MDBM.jl")
using Reexport
@reexport using .MDBM

using PyPlot;
pygui(true);

In [2]:
function foo(x,y)
    x^2.0+y^2.0-4.0^2.0
end
function c(x,y)
    x-y
end

ax1=Axis([-5,-2.5,0,2.5,5],"x")
ax2=Axis(-5:2:5.0,"b")

mymdbm=MDBM_Problem(foo,[ax1,ax2],constraint=c)
iteration=6 #number of refinements (resolution doubling)
solve!(mymdbm,iteration)


#evaluated points
x_eval,y_eval=getevaluatedpoints(mymdbm)

#solution points
x_sol,y_sol=getinterpolatedsolution(mymdbm)

fig = figure(1);clf()
scatter(x_eval,y_eval,s=5)
scatter(x_sol,y_sol,s=5)

myDT1=connect(mymdbm);
for i in 1:length(myDT1)
    dt=myDT1[i]
    P1=getinterpolatedsolution(mymdbm.ncubes[dt[1]],mymdbm)
    P2=getinterpolatedsolution(mymdbm.ncubes[dt[2]],mymdbm)
    plot([P1[1],P2[1]],[P1[2],P2[2]], color="k")
end 

In [85]:
using LinearAlgebra
axes=[-2:2,-2:2,-2:2]
fig = figure(3);clf()


#Sphere1
fS1=(x...)->norm([x...],2.0)-1
Sphere1mdbm=MDBM_Problem(fS1,axes)
solve!(Sphere1mdbm,4)
a_sol,b_sol,c_sol=getinterpolatedsolution(Sphere1mdbm)
plot3D(a_sol,b_sol,c_sol,linestyle="", marker=".", markersize=1);


#Sphere2
fS2=(x...)->norm([x...] .- 0.5,2.0) -1.0
Sphere2mdbm=MDBM_Problem(fS2,axes)
solve!(Sphere2mdbm,4)
a_sol,b_sol,c_sol=getinterpolatedsolution(Sphere2mdbm)
plot3D(a_sol,b_sol,c_sol,linestyle="", marker=".", markersize=1);

fS12=(x...)->[fS1(x...),fS2(x...)]
Intersectmdbm=MDBM_Problem(fS12,axes)
solve!(Intersectmdbm,6)

#Intersection
a_sol,b_sol,c_sol=getinterpolatedsolution(Intersectmdbm)
plot3D(a_sol,b_sol,c_sol,color="k",linestyle="", marker=".", markersize=2);

In [111]:
fig = figure(4);clf()
function mandelbrot(x,y)    
    c=x+y*1im
    z=Complex(0)
    k=0
    maxiteration=1000
    while (k<maxiteration && abs(z)<4.0)
            z=z^2+c
            k=k+1
        end
    return abs(z)-2.0 #(k==maxiteration)# && abs(z)<2.0)#abs(z)<2.0
end

Mandelbrotmdbm=MDBM_Problem(mandelbrot,[-5:2,-2:2])
solve!(Mandelbrotmdbm,9)
a_sol,b_sol=getinterpolatedsolution(Mandelbrotmdbm)
plot(a_sol,b_sol,linestyle="", marker=".", markersize=1);

0.44406845960638414

In [ ]:
plot3D(a_sol,b_sol,c_sol,linestyle="", linewidth=3, marker=".", markersize=0.5);

In [3]:
function f(a,b)
    a^2.0+b^2.0-2.0^2.0
end
function c(a,b)
    a-b
end

ax1=Axis([-5,-2.5,0,2.5,5],"a")
ax2=Axis(-5:2:5.0,"b")

mymdbm=MDBM_Problem(f,[ax1,ax2],constraint=c)
interpolate!(mymdbm,interpolationorder=1)

In [4]:
for k=1:3
    @time refine!(mymdbm)
    @time interpolate!(mymdbm,interpolationorder=1)
    println("------")
end
println("+++++++++++++")


#evaluated points
a_eval,b_eval=getevaluatedpoints(mymdbm)


#solution points
a_sol,b_sol=getinterpolatedsolution(mymdbm)

fig = figure(1)
scatter(a_eval,b_eval)

# scatter(a_sol,b_sol,xlabel = ax1.name,ylabel = ax2.name,size = (500, 500),
#      xticks = mymdbm.axes[1].ticks , yticks = mymdbm.axes[2].ticks, gridalpha=0.8)
scatter(a_sol,b_sol);


# @time myDT1=DTconnect(mymdbm);
# triplot(a_sol,b_sol, triangles=myDT1)

  0.003232 seconds (590 allocations: 40.625 KiB)
  0.000564 seconds (2.38 k allocations: 1.471 MiB)
------
  0.000332 seconds (1.19 k allocations: 94.375 KiB)
  0.000512 seconds (5.27 k allocations: 3.209 MiB)
------
  0.000489 seconds (2.32 k allocations: 187.000 KiB)
  0.001028 seconds (10.73 k allocations: 6.644 MiB)
------
+++++++++++++


In [19]:
@time checkneighbour!(mymdbm)
@time interpolate!(mymdbm,interpolationorder=1)

#evaluated points
a_eval=[x.callargs[1] for x in mymdbm.fc.fvalarg]
b_eval=[x.callargs[2] for x in mymdbm.fc.fvalarg]

#solution points
a_sol,b_sol=getinterpolatedsolution(mymdbm)


fig = figure(2)
# cla())
# clf()
scatter(a_eval,b_eval)

# scatter!(a_sol,b_sol,xlabel = ax1.name,ylabel = ax2.name,size = (500, 500),
#      xticks = mymdbm.axes[1].ticks , yticks = mymdbm.axes[2].ticks, gridalpha=0.8)
scatter(a_sol,b_sol);


  0.001527 seconds (6.98 k allocations: 755.359 KiB)
  0.000716 seconds (4.70 k allocations: 3.061 MiB)


In [20]:
println(mymdbm.fc.memoryacc[1])
println(length(mymdbm.fc.fvalarg))

939
197


In [5]:
using LinearAlgebra
mymdbm=MDBM_Problem((x...)->norm([x...] .- 0.4,2.7)-1.9,[-2:2,-2:2,-2:2])

interpolate!(mymdbm,interpolationorder=1)
for k=1:4
    @time refine!(mymdbm)
    @time interpolate!(mymdbm,interpolationorder=1)
    println("------")
end
@time checkneighbour!(mymdbm)
@time interpolate!(mymdbm,interpolationorder=1)

println("+++++++++++++")

#evaluated points
a_eval,b_eval,c_val=getevaluatedpoints(mymdbm)

#solution points
a_sol,b_sol,c_sol=getinterpolatedsolution(mymdbm)


fig = figure(3);clf()
# scatter(a_eval,b_eval,c_eval,markersize=1)

plot3D(a_sol,b_sol,c_sol,linestyle="", linewidth=3, marker=".", markersize=0.5);

  0.005764 seconds (13.28 k allocations: 1.121 MiB)
  0.023384 seconds (57.83 k allocations: 31.852 MiB, 63.51% gc time)
------
  0.010327 seconds (67.59 k allocations: 5.499 MiB)
  0.047467 seconds (246.28 k allocations: 135.902 MiB, 32.94% gc time)
------
  0.060598 seconds (334.03 k allocations: 27.519 MiB, 11.77% gc time)
  0.187887 seconds (983.36 k allocations: 543.531 MiB, 25.52% gc time)
------
  0.249241 seconds (1.52 M allocations: 126.212 MiB, 11.07% gc time)
  1.175197 seconds (4.08 M allocations: 2.202 GiB, 22.97% gc time)
------
  1.520947 seconds (8.11 M allocations: 3.782 GiB, 24.62% gc time)
  0.377975 seconds (1.91 M allocations: 1.097 GiB, 30.74% gc time)
+++++++++++++


In [50]:
using LinearAlgebra
mymdbm=MDBM_Problem((x...)->norm([x...].+ 1.5,1.7)+sin(x[1]*5)-2.0,[-2:2,-2:2])

interpolate!(mymdbm,interpolationorder=1)
for k=1:5
    refine!(mymdbm)
    interpolate!(mymdbm,interpolationorder=1)
end
checkneighbour!(mymdbm)
interpolate!(mymdbm,interpolationorder=1)

#evaluated points
a_eval,b_eval=getevaluatedpoints(mymdbm)

a_sol,b_sol=getinterpolatedsolution(mymdbm)

fig = figure(4)
plot(a_eval,b_eval,linestyle="",  marker=".",markersize=1)
plot(a_sol,b_sol,linestyle="",  marker=".",markersize=1);

In [51]:
# # extension with the same resolution:
# axesextend!(mymdbm,1,prepend=mymdbm.axes[1].ticks[1:end-1] .+(mymdbm.axes[1].ticks[1]- mymdbm.axes[1].ticks[end]));
# axesextend!(mymdbm,2,prepend=mymdbm.axes[2].ticks[1:end-1] .+(mymdbm.axes[2].ticks[1]- mymdbm.axes[2].ticks[end]));

#extension with different reselution
axesextend!(mymdbm,1,prepend=-6.2:0.2:-2.2);
axesextend!(mymdbm,2,prepend=-6.2:0.1:-2.2,append=2.2:0.2:3);

In [52]:
checkneighbour!(mymdbm)
interpolate!(mymdbm,interpolationorder=1)

#evaluated points
a_eval,b_eval=getevaluatedpoints(mymdbm)

#solution points
a_sol,b_sol=getinterpolatedsolution(mymdbm)


# scatter(a_eval,b_eval,markersize=1)
# scatter!(a_sol,b_sol,size = (800, 800),markersize=2,
#      xticks = mymdbm.axes[1].ticks , yticks = mymdbm.axes[2].ticks, gridalpha=0.8)

fig = figure(4)
clf()
plot(a_eval,b_eval,linestyle="",  marker=".",markersize=1)
plot(a_sol,b_sol,linestyle="",  marker=".",markersize=1);

In [53]:
for k=1:2
    refine!(mymdbm)
    interpolate!(mymdbm,interpolationorder=1)
end
checkneighbour!(mymdbm)
interpolate!(mymdbm,interpolationorder=1)

#evaluated points
a_eval,b_eval=getevaluatedpoints(mymdbm)

#solution points
a_sol,b_sol=getinterpolatedsolution(mymdbm)


# scatter(a_eval,b_eval,markersize=1)
# scatter!(a_sol,b_sol,size = (800, 800),markersize=2,
#  xticks = mymdbm.axes[1].ticks , yticks = mymdbm.axes[2].ticks, gridalpha=0.8)

fig = figure(4)
clf()
plot(a_eval,b_eval,linestyle="",  marker=".",markersize=1)
plot(a_sol,b_sol,linestyle="",  marker=".",markersize=1);

In [54]:
using LinearAlgebra

ax1=Axis(-5:3.0,"a")
ax2=Axis(-5:3.0,"b")

mymdbm=MDBM_Problem((x...)->[],[ax1,ax2],constraint=(x...) -> -(norm([x...].+ 1.5,1.7)+sin(x[1]*5)-2.0))
# mymdbm=MDBM_Problem((x...) -> -maximum([0.0,-(norm([x...].+ 1.5,1.7)+sin(x[1]*5)-2.0)]),[ax1,ax2])
interpolate!(mymdbm,interpolationorder=1)
for k=1:4
    refine!(mymdbm)
    interpolate!(mymdbm,interpolationorder=1)
end
checkneighbour!(mymdbm)
interpolate!(mymdbm,interpolationorder=1)

#solution points
a_sol,b_sol=getinterpolatedsolution(mymdbm)

F_sol=map((x,y)->x*x-y*y,a_sol,b_sol)
# scatter(a_sol,b_sol,F_sol,size = (500, 500))


fig = figure(5)
plot3D(a_sol,b_sol,F_sol,linestyle="",  marker=".",markersize=4);

DimensionMismatch: DimensionMismatch("cannot broadcast array to have fewer dimensions")

QXcbConnection: XCB error: 148 (Unknown), sequence: 7763, resource id: 0, major code: 140 (Unknown), minor code: 20
QXcbConnection: XCB error: 148 (Unknown), sequence: 7764, resource id: 0, major code: 140 (Unknown), minor code: 20
QXcbConnection: XCB error: 148 (Unknown), sequence: 7765, resource id: 0, major code: 140 (Unknown), minor code: 20


In [13]:
using LinearAlgebra

ax1=Axis(-5:0.3:3.0,"a")
ax2=Axis(-5:0.25:3.0,"b")
ax3=Axis(-5:0.5:3.0,"b")


mymdbm=MDBM_Problem((x...) -> -(norm([x...],2.505)-1.0^(2.505)),[ax1,ax2,ax3],constraint=(x...)->[x...])
interpolate!(mymdbm,interpolationorder=1)
for k=1:2
    refine!(mymdbm)
    interpolate!(mymdbm,interpolationorder=1)
end
checkneighbour!(mymdbm)

interpolate!(mymdbm,interpolationorder=1)

#solution points
# a_sol,b_sol=getinterpolatedsolution(mymdbm)
a_sol,b_sol,c_sol=getinterpolatedsolution(mymdbm);

In [14]:
fig = figure(6)
clf()

@time myDT1=connect(mymdbm);
# plot3D(a_sol,b_sol,c_sol,linestyle="",  marker=".",markersize=6)
plot(a_sol,b_sol,linestyle="",  marker=".",markersize=8)

 for i in 1:length(myDT1)
     dt=myDT1[i]
     P1=getinterpolatedsolution(mymdbm.ncubes[dt[1]],mymdbm)
     P2=getinterpolatedsolution(mymdbm.ncubes[dt[2]],mymdbm)
      plot3D([P1[1],P2[1]],[P1[2],P2[2]],[P1[3],P2[3]], color="k")
#       plot([P1[1],P2[1]],[P1[2],P2[2]])
  end 


  0.318272 seconds (857.27 k allocations: 55.343 MiB, 4.54% gc time)


In [15]:
fig = figure(7)
clf()
myDT2=triangulation(myDT1)


plot_trisurf(a_sol,b_sol,c_sol,triangles=myDT2);